### XY Dataset Regression Problems



Imports

In [1]:
import numpy as np

import torch

from rich.pretty import pprint

import matplotlib.pyplot as plt

from pyml.ds import XYDataset
from pyml.ds.generic import GenericDataset, generic_split_dataset
from pyml.mlp import MLP
from pyml.supervision import hypervise_mlp
from pyml.ds.util import dataset_min_max, dataset_mean_std

import ipywidgets as widgets

torch.manual_seed(50)

##### $f_{x}$

_1. create the dataset and split it into two_ `GenericDataset`_s; one for training, the other for testing._

_2. provide the loss-function, optimizer class, datasets; model-hidden-layers and hidden-layers' size; activation, dropouts, thresholds, learning-rate, batch-size and the rest--if any--of the keyword-arguments for the optimizer-instantiation, using the _`hypervise_mlp`_ function._

In [ ]:
@widgets.interact_manual(
    fx={"x": lambda x: x, "2*x": lambda x: 2*x, "x**2": lambda x: x**2},
    domain_start = widgets.IntText(value=-100),
    domain_end = widgets.IntText(value=100),
    H=widgets.IntText(value=10),
    Hn=widgets.IntText(value=1),
    interval=widgets.FloatText(value=0.1),
    epochs=widgets.IntText(value=50),
    lr=widgets.FloatText(value=1e-4),
    loss_fn={"mse": torch.nn.MSELoss, "l1": torch.nn.L1Loss, "smoothl1": torch.nn.SmoothL1Loss, "huber": torch.nn.HuberLoss},
    activation={"relu": torch.nn.ReLU(), "leaky-relu": torch.nn.LeakyReLU()},
    optim={"sgd": torch.optim.SGD, "adam": torch.optim.Adam}
)

def run(
    fx = "x",
    domain_start=-100,
    domain_end=100,
    H=10,
    Hn=2,
    interval=0.1,
    epochs=50,
    lr=1e-4,
    loss_fn="l1",
    activation="relu",
    optim="sgd"
):
    xy_dataset = XYDataset(
        fn=fx,
        domain=[(domain_start, domain_end)],
    )

    xy_train_ds, xy_test_ds = generic_split_dataset(
        dataset=xy_dataset,
        test_split=0.2,
        transforms=None,
        feat_transforms=None,
        targ_transforms=None
    )

    xy_model, results= hypervise_mlp(
        loss_fn_cls=loss_fn,
        optim_cls=optim,
        train_dataset=xy_train_ds,
        test_dataset=xy_test_ds,
        Hn=Hn,
        H=H,
        activation=activation,
        dropouts=None,
        thresholds=None,
        dtype=torch.float32,
        track_interval=interval,
        epochs=epochs,
        batch_size=256,
        shuffle=True,
        lr=lr
    )

    results[["interval-loss", "loss", "interval-loss-test", "loss-test"]].plot()

interactive(children=(Dropdown(description='fx', options={'x': <function <lambda> at 0x0000024014C79080>, '2*x…